In [1]:
import numpy
from matplotlib import pyplot

%matplotlib inline

In [2]:
#SET parameters
L = 0.01
nx, ny = 21, 21
dx = L / (nx - 1)
dy = L / (ny - 1)
x = numpy.linspace(0.0, L ,num=nx)
y = numpy.linspace(0.0, L, num=ny)

alpha = 1e-4

#Set initial solution 
T0 = numpy.full((ny, nx), 20.0)
T0[:,0] = 100.0#bottom
T0[0,:] = 100.0#left

In [3]:
def btcs(T0, nt, dt, delta,alpha):
    sigma = alpha * dt / delta**2
    #Create LHS operator
    A = lhs_operator()
    T = map_2d_to_1d(T0)
    #integrate in time
    for n in range(nt):
        b = rhs_vector()
        T = numpy.linalg.solve(A,b)
    return map_1d_to_2d(T)

In [6]:
def lhs_operator(M,N):
    A = numpy.zeros((M * N, M * N))
    for j in range(N):
        for i in range(M):
            I = j * M + i
            west = I - 1
            east = I + 1
            south = I - M
            north = I + M
            if i == 0 and j == 0: #bottom-left corner (interior)
                A[I,I] = 1 / sigma + 4
                A[I, east] = -1.0
                A[I, north] = -1.0
            elif i == M - 1 and j == 0:#bottom-right corner
                A[I,I] = 1 / sigma + 3 #(1 / sigma + 4 - 1)
                A[I,west] = -1.0
                A[I, north] = -1.0
            elif i == 0 and j ==M - 1: #top-left corner(interior)
                A[I,I] = 1 / sigma + 3
                A[I, east] = -1.0
                A[I, south] = -1.0
            elif i == M -1 and j == N - 1: #top-right corner
                A[I,I] = 1/ sigma + 2
                A[I, west] = -1.0
                A[I, south] = -1.0
            elif j == 0: #first row of interior points
                A[I,I] = 1 / sigma + 4
                A[I,west] = -1.0
                A[I,east] = -1.0
                A[I, north] = -1.0
            elif j == N -1:#last row of interior points
                A[I,I] = 1 / sigma + 3
                A[I,south] = -1.0
                A[I, west] = -1.0
                A[I, east] = -1.0
            elif i == 0: # first column of interior points
                A[I, I] = 1.0 / sigma + 4.0
                A[I, west] = -1.0
                A[I, east] = -1.0
                A[I, north] = -1.0
            elif i == M - 1: #last column of interior points
                A[I, I] = 1.0 / sigma + 3.0
                A[I, south] = -1.0
                A[I, west] = -1.0
                A[I, east] = -1.0
            else:
                A[I,I] = 1/sigma + 4
                A[I,south] = -1.0
                A[I,west] = -1.0
                A[I,east] = -1.0
                A[I,north] = -1.0
    return A 
            


In [7]:
def map_1d_to_2d(T_1d,M, N,Tb):
    T = numpy.zeros(N+2,M+2)
    T[1:-1.1:-1] = T_1d.reshape((N,M))
    #uSE dIRICHLET CONDITION AT LEFT AND BOTTOM BOUNDARIES
    T[:,0] = Tb
    T[0.:] = Tb
    return T

In [8]:
def map_2d_to_1d(T):
    T_1d = T[1:-1,1:-1].flatten()
    return T_1d

In [ ]:
def